## Recommender System - Marketplace Matching

In this notebook, we will: 
- Clean textual data from user-input verbatim posts
- Use a Word2Vec model to calculate document similarities 
- Sort the most similar user input to our training data in order to recommend similar products 
- Save this model in a format that allows us to refresh the testing data

The goal of this project is to create a recommender system to help Pangeans find the right "project" for them, given their profile information. Here, we are using legacy data that is from Pangea V2, when Pangeans were allowed to post services and requests, as well as purchase items on the platform. We are using the User-Inputted Titles to suggest similar services, or in V3, similar "projects". 

In [1]:
'''Importing Libraries'''
import numpy as np
import pandas as pd

import sys
from pandas import DataFrame

import json
from pandas.io.json import json_normalize
import csv

import matplotlib.pyplot as plt
%matplotlib inline 
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from gensim import corpora
from collections import defaultdict
from pprint import pprint

from gensim.models.word2vec import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import common_texts

import re
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
nltk.download('stopwords') ###
from nltk.tokenize import RegexpTokenizer

import gensim 
from gensim.models import KeyedVectors
import gensim.downloader as api

from operator import itemgetter, attrgetter

from gensim.models.doc2vec import Doc2Vec

import os, sys

from operator import add

from sklearn.preprocessing import normalize

import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from adjustText import adjust_text

from sklearn.manifold import TSNE

import pickle

import json
from pprint import pprint
from pandas.io.json import json_normalize

import operator

import csv

import string

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/angelateng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
'''Loading the model'''
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True)

In [24]:
'''Vectorize and store existing titles in legacy Pangea database'''
def vectorize_and_store_existing_titles(): 
    raw = pd.read_csv("allPostData.csv", header=0);
    titles = raw['title'];
    post_titles = [title for title in titles];
    post_titles = set(post_titles); 
    tokens = [[word for word in title.lower().split()] for title in post_titles];
    clean_words = [[word.translate(str.maketrans('', '', string.punctuation)) for word in title] for title in tokens]
    #clean_words = [[word for word in title if word.isalpha()] for title in tokens];
    tokenizer = RegexpTokenizer(r'\w+')
    
    stoplist = set(stopwords.words('english'));
    titles_nostopwords = [[word for word in title if word not in stoplist] for title in clean_words];   
    filtered_word_list = [[word for word in title if word in model.vocab] for title in titles_nostopwords];
    dictionary = dict(zip(post_titles, filtered_word_list))
    vectorized_titles = pd.DataFrame(columns=["Titles", "Vectors"])
    for title in post_titles: 
        word_vecs = [model[word] for word in dictionary[title]]
        if len(word_vecs) == 0:
            title_vec = [np.zeros(300)]
        else: 
            title_vec = normalize(sum(word_vecs).reshape(1, -1))
        vectorized_titles = vectorized_titles.append({'Titles': title, 'Vectors': title_vec}, ignore_index=True)
    vectorized_titles.to_pickle("/Users/angelateng/Dropbox/AMBER/SharpestMinds/vectorized_titles.pkl")
    return(vectorized_titles)


In [14]:
raw = pd.read_csv("allPostData.csv", header=0);
titles = raw['title'];
vectorize_and_store_existing_titles()

,Titles,Vectors
0,Back massage,"[[0.046960913, -0.02392877, 0.020748543, 0.058..."
1,Help with case prep for consulting,"[[-0.008480151, 0.106523395, -0.014692882, 0.0..."
2,Portraits,"[[0.061697837, 0.10825166, -0.00341792, 0.0768..."
3,Whatever,"[[0.043075226, -0.060968015, 0.0037000517, 0.0..."
4,Birthday Planner Wanted,"[[0.03388815, -0.011697535, 0.037889402, 0.047..."
5,Goals,"[[-0.067896895, 0.07709787, 0.051081307, 0.043..."
6,Lessons On App Prototyping,"[[0.020253977, -0.031405043, -0.050369438, 0.0..."
7,Chem tutor,"[[-0.028732741, 0.04354651, 0.0032809014, 0.11..."
8,Physics Tutor,"[[0.054761842, -0.045980215, 0.08199476, 0.047..."
9,LaTex Tutor,"[[0.0619012, -0.0149769895, -0.07452506, 0.088..."


In [26]:
'''Print a dataframe to show how these titles and their vectors will be saved'''
'''Also note that we are saving the df with the original raw (not cleaned) titles'''
vectorize_and_store_existing_titles()

,Titles,Vectors
0,Back massage,"[[0.046960913, -0.02392877, 0.020748543, 0.058..."
1,Help with case prep for consulting,"[[-0.008480151, 0.106523395, -0.014692882, 0.0..."
2,Portraits,"[[0.061697837, 0.10825166, -0.00341792, 0.0768..."
3,Whatever,"[[0.043075226, -0.060968015, 0.0037000517, 0.0..."
4,Birthday Planner Wanted,"[[0.03388815, -0.011697535, 0.037889402, 0.047..."
5,Goals,"[[-0.067896895, 0.07709787, 0.051081307, 0.043..."
6,Lessons On App Prototyping,"[[0.020253977, -0.031405043, -0.050369438, 0.0..."
7,Chem tutor,"[[-0.028732741, 0.04354651, 0.0032809014, 0.11..."
8,Physics Tutor,"[[0.054761842, -0.045980215, 0.08199476, 0.047..."
9,LaTex Tutor,"[[0.0619012, -0.0149769895, -0.07452506, 0.088..."


In [17]:
'''Read a JSON file when a new user pings with a request'''
with open('firstPost.json') as fresh_data:
    user_post = json.load(fresh_data)

'''Vectorize each new title as a user/student/company creates a new post'''
def vectorize_new_title(user_post):
    #ranked_titles_load = pd.read_csv("./ranked_titles.csv")
    json_df = pd.DataFrame.from_dict(json_normalize(user_post), orient='columns')
    title = json_df["title"][0] #--> added [0] because json_df["title"] was still in pd.Series form
    json_tokens = [word for word in title.lower().split()]
    json_clean_words = [word for word in json_tokens if word.isalpha()]
    stoplist = set(stopwords.words('english'));
    json_titles_nostopwords = [word for word in json_clean_words if word not in stoplist]   
    json_preprocessed = [word for word in json_titles_nostopwords if word in model.vocab]   
    json_title_vectors = {}
    json_vectorized_title_df = pd.DataFrame(columns=["Titles", "Vectors"])
    json_word_vecs = [model[word] for word in json_preprocessed]
    if len(json_preprocessed) == 0:
            json_title_vec = [np.zeros(300)]
    else: 
        json_title_vec = normalize(sum(json_word_vecs).reshape(1, -1))
    json_vectorized_title_df = json_vectorized_title_df.append({'Titles': title, 'Vectors': json_title_vec}, ignore_index = True)
    #export_csv = json_vectorized_title_df.to_csv (r'/Users/angelateng/Dropbox/AMBER/SharpestMinds/ranked_titles.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path
    if not os.path.isfile('/Users/angelateng/Dropbox/AMBER/SharpestMinds/ranked_titles.csv'):
        json_vectorized_title_df.to_csv (r'/Users/angelateng/Dropbox/AMBER/SharpestMinds/ranked_titles.csv', index = None, header=True)
    else:
        json_vectorized_title_df.to_csv (r'/Users/angelateng/Dropbox/AMBER/SharpestMinds/ranked_titles.csv', mode='a', index = None, header=False)
    
    return(json_vectorized_title_df)

In [18]:
'''Append new user-post title to csv'''
vectorized_title = vectorize_new_title(user_post)
print(vectorized_title)

                  Titles                                            Vectors
0  Teach Me How To Cook!  [[0.042356346, -0.013683017, 0.07774045, 0.028...


In [19]:
'''Load the current titles in the Pangea database, and then rank them by similarity to the latest user query'''
def rank_existing_titles(vectorized_title):
    ranked_titles = {}
    other_titles = pd.read_pickle("./vectorized_titles.pkl")
    for index,row in other_titles.iterrows():
        ranked_titles[row['Titles']] = sum(row['Vectors'][0]*vectorized_title['Vectors'][0][0]) # --> did the dot product using sum() and * because np.dot was behaving weirdly for some reason. Now it seems to work! 
    sorted_title_vecs = sorted(ranked_titles.items(), key=operator.itemgetter(1), reverse=True)
    return(sorted_title_vecs)

#should this also be named ranked titles? or should we rename this line? since we're 
#already loading them in the cell above ??


In [20]:
rank_existing_titles(vectorized_title)

[('Teach Me How To Cook!', 0.7957720306183091),
 ('Teach you how to shuffle', 0.7285079273577821),
 ('Teach me how to play piano', 0.7146377012308847),
 ('Teach how to make tea', 0.6914844676543908),
 ('Teach how to prepare steaks ', 0.6608135595374733),
 ('Teach Me How To Solve Rubiks Cube', 0.6324938975573104),
 ('Teach me meme culture', 0.6134594880758186),
 ('Teach me how to cook quick meals!', 0.60577990586944),
 ('Self-Defense Lessons', 0.5882213850279641),
 ('Learn To Juggle!', 0.5785073572219517),
 ('Orgo lesson ', 0.5588066486525349),
 ('Math Tutor', 0.5531746304923217),
 ('Math tutor', 0.5531746304923217),
 ('Cooking Lessons', 0.5473080828668344),
 ('Cooking lessons', 0.5473080828668344),
 ('Cooking Lessons ', 0.5473080828668344),
 ('Help me learn Pop/Stars dance', 0.5464415329543044),
 ('Learn To Harmonize', 0.5445825279987275),
 ('Prototyping/Wireframing Lessons', 0.5328233039076622),
 ('Amharic lessons ', 0.5328233039076622),
 ('Fortnite Lessons', 0.5328233039076622),
 ('M

In [21]:
'''Final function call API that puts together the prior 3 functions in a neat mega-function'''
def generate_recommendations(user_post):
    title = user_post["title"]
    vectorized_title = vectorize_new_title(user_post)
    ranked_titles = rank_existing_titles(vectorized_title)    
    with open("./ranked_titles.csv", "w", newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        for title in ranked_titles:
            wr.writerow([ranked_titles, title])
    return(ranked_titles)

#should this still be ranked titles.csv???


In [22]:
generate_recommendations(user_post)

[('Teach Me How To Cook!', 0.7957720306183091),
 ('Teach you how to shuffle', 0.7285079273577821),
 ('Teach me how to play piano', 0.7146377012308847),
 ('Teach how to make tea', 0.6914844676543908),
 ('Teach how to prepare steaks ', 0.6608135595374733),
 ('Teach Me How To Solve Rubiks Cube', 0.6324938975573104),
 ('Teach me meme culture', 0.6134594880758186),
 ('Teach me how to cook quick meals!', 0.60577990586944),
 ('Self-Defense Lessons', 0.5882213850279641),
 ('Learn To Juggle!', 0.5785073572219517),
 ('Orgo lesson ', 0.5588066486525349),
 ('Math Tutor', 0.5531746304923217),
 ('Math tutor', 0.5531746304923217),
 ('Cooking Lessons', 0.5473080828668344),
 ('Cooking lessons', 0.5473080828668344),
 ('Cooking Lessons ', 0.5473080828668344),
 ('Help me learn Pop/Stars dance', 0.5464415329543044),
 ('Learn To Harmonize', 0.5445825279987275),
 ('Prototyping/Wireframing Lessons', 0.5328233039076622),
 ('Amharic lessons ', 0.5328233039076622),
 ('Fortnite Lessons', 0.5328233039076622),
 ('M

In [27]:
#sanity check
df = pd.DataFrame(generate_recommendations(user_post))
df.columns = ["Title", "Similarity Score"]
print(df)

                                 Title  Similarity Score
0                Teach Me How To Cook!          0.795772
1             Teach you how to shuffle          0.728508
2           Teach me how to play piano          0.714638
3                Teach how to make tea          0.691484
4         Teach how to prepare steaks           0.660814
5    Teach Me How To Solve Rubiks Cube          0.632494
6                Teach me meme culture          0.613459
7    Teach me how to cook quick meals!          0.605780
8                 Self-Defense Lessons          0.588221
9                     Learn To Juggle!          0.578507
10                        Orgo lesson           0.558807
11                          Math Tutor          0.553175
12                          Math tutor          0.553175
13                     Cooking Lessons          0.547308
14                     Cooking lessons          0.547308
15                    Cooking Lessons           0.547308
16       Help me learn Pop/Star